In [22]:
import pyaudio
import wave
import os
from transformers import pipeline
import pywhisper
import torch
import av
import torch
import numpy as np
from transformers import LlavaNextVideoProcessor, LlavaNextVideoForConditionalGeneration

### Transcripción y resumen de audio

In [6]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Usando dispositivo: {device}")
print("GPU disponible?:", torch.cuda.is_available())

def transcribir_audio(archivo_audio):
    modelo = pywhisper.load_model("base", device=device)  # Puedes ajustar el modelo según tus necesidades
    resultado = modelo.transcribe(archivo_audio)
    return ' '.join(dic['text'] for dic in resultado['segments'])

Usando dispositivo: cpu
GPU disponible?: False


In [7]:
summarizer = pipeline("summarization", model="facebook/bart-large-cnn")
def summarize_long_text(text, chunk_size=512):
    """Summarizes a long text by splitting it into chunks."""
    chunks = [text[i:i + chunk_size] for i in range(0, len(text), chunk_size)]
    summary_chunks = []

    for chunk in chunks:
        summary_chunks.append(summarizer(chunk, max_length=130, min_length=30, do_sample=False)[0]['summary_text'])

    return " ".join(summary_chunks)

Device set to use cpu


In [13]:
transcripcion = transcribir_audio("./grabaciones/podcast.mp3")
print(transcripcion)

 El tema de los buscadores de cada vez, que es una tema muy heavy, que se que no podés  mostrar en YouTube tal cual lo viviste, pero ¿qué es esto? ¿Qué significa hasta  los buscadores de cada vez para la gente que no lo sabe?  Platzard, que antes se llamaba los tulipanes negros, que lo verán, son un grupo que  medio pena no haberlo explicado mejor en la pieza en el capítulo.  Ahora puedes espalhar de aquí.  Es algo muy sencillo, que son civiles, no son militares.  No, si hay, tenemos un gráfico, un rótulo que hablamos que son voluntarios como forma  de esa, nuestra idea es explicar que son civiles, porque para mí me parecía de hecho una  parte clave de la historia, que son civiles, pasas que van vestidos como militares,  van de camuflaje y con su casco, con su será levantivalas, por el objetivo metácelo  en el labor en zona guerra.  Entonces aglarar que son civiles, que me parece todavía más fuertes civiles, voluntarios  que se meten en las zonas, campos repletos de minas, el protagoni

In [14]:
resumen = summarize_long_text(transcripcion)
print(resumen)

Your max_length is set to 130, but your input_length is only 4. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=2)


"Platzard" es un grupo que  medio pena no haberlo explicado mejor. Es algo muy sencillo, que son civiles, no son militares. No, if hay, tenemos un gráfico, un rótulo q" "Ue hablamos que son voluntarios como forma  de esa, nuestra idea es explicar that son civiles, porque para mí me parece de hecho una  parte clave of la historia, que sonCiviles, pasas que van vestidos como militares,  van de camuflaje" Parece brutal la visión que tiene como habla de las almas. No hace distinción  de Ukrainianos y de rusos. Ellos tienen una centraita, como una sede, que es u u "No sé, a ver, junto  a los restos humanos, había un femur" "Pues él decía que no sé if eran rusos" Chamusca o esa saber, eso era lo que había sido el frente de dos años atrás, era todo  chamusca or de los misiles, los árboles, negros, los that quedaban, o sea, el manto de  nieve porque estamos en invierno, pero with trozos de tanques, con trozos of cañones,  sí, era una estampa. Claro que, if hablamos de los olores, ese es realme

### Transcripción de grabación

In [2]:
FRAMES=1024
FORMAT = pyaudio.paInt16
CHANNELS = 1
RATE = 44100
RUTA_GRABACIONES= "./grabaciones"
os.makedirs(RUTA_GRABACIONES, exist_ok=True)
p = pyaudio.PyAudio()

stream = p.open(
                format=FORMAT,
                channels=CHANNELS,
                rate=RATE,
                input=True,
                frames_per_buffer=FRAMES
                )
print(f"Comienzo de la grabación...")
frames=[]
segundos=10
for j in range(0,int(RATE/FRAMES*segundos)):
    data=stream.read(FRAMES)
    frames.append(data)

print(f"Fin de la grabación...")
stream.stop_stream()
stream.close()
filename = os.path.join(RUTA_GRABACIONES, f"grabacion.wav")
with wave.open(filename, 'wb') as wf:
    wf.setnchannels(CHANNELS)
    wf.setsampwidth(2)  
    wf.setframerate(RATE)
    wf.writeframes(b''.join(frames))

p.terminate()

print("Grabacion completadas.")

Comienzo de la grabación...
Fin de la grabación...
Grabacion completadas.


In [8]:
transcripcion = transcribir_audio("./grabaciones/grabacion.wav")
print(transcripcion)

c:\Users\rafae\Desktop\AINE_AUDIO_SUMMARIZE\.venv\Lib\site-packages\pywhisper\transcribe.py:78: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 Pero a un altar de mocha...  ¡Octía mierda!  Que estoy cambiando la grabación del otro día.


In [9]:
resumen = summarize_long_text(transcripcion)
print(resumen)

Your max_length is set to 130, but your input_length is only 40. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=20)


Pero a un altar de mocha...  ¡Octía mierda!  Que estoy cambiando la grabación del otro día.


### Transcripción de video

In [32]:
model_id = "llava-hf/LLaVA-NeXT-Video-7B-hf"

model = LlavaNextVideoForConditionalGeneration.from_pretrained(
    model_id, 
    torch_dtype=torch.float16, 
    low_cpu_mem_usage=True, 
).to(device)

processor = LlavaNextVideoProcessor.from_pretrained(model_id)

def read_video_pyav(container, indices):
    '''
    Decode the video with PyAV decoder.
    Args:
        container (`av.container.input.InputContainer`): PyAV container.
        indices (`List[int]`): List of frame indices to decode.
    Returns:
        result (np.ndarray): np array of decoded frames of shape (num_frames, height, width, 3).
    '''
    frames = []
    container.seek(0)
    start_index = indices[0]
    end_index = indices[-1]
    for i, frame in enumerate(container.decode(video=0)):
        if i > end_index:
            break
        if i >= start_index and i in indices:
            frames.append(frame)
    return np.stack([x.to_ndarray(format="rgb24") for x in frames])


# define a chat history and use `apply_chat_template` to get correctly formatted prompt
# Each value in "content" has to be a list of dicts with types ("text", "image", "video") 
conversation = [
    {

        "role": "user",
        "content": [
            {"type": "text", "text": "Describe el video. Debes responder en español."},
            {"type": "video"},
            ],
    },
]

prompt = processor.apply_chat_template(conversation, add_generation_prompt=True)

def transcribir_video(video_path):
  container = av.open(video_path)
  total_frames = container.streams.video[0].frames
  indices = np.arange(0, total_frames, total_frames / 8).astype(int)
  clip = read_video_pyav(container, indices)
  inputs_video = processor(text=prompt, videos=clip, padding=True, return_tensors="pt").to(model.device)

  output = model.generate(**inputs_video, max_new_tokens=100, do_sample=False)
  
  transcription = processor.decode(output[0][2:], skip_special_tokens=True)
  return transcription

Loading checkpoint shards: 100%|██████████| 3/3 [00:20<00:00,  6.92s/it]


In [33]:
video_path = "./grabaciones/papa.mp4"
transcripcion = transcribir_video(video_path)

In [34]:
print(transcripcion)

ER: 
Describe el video. Debes responder en español. ASSISTANT: Lo sient


In [35]:
resumen = summarize_long_text(transcripcion)
print(resumen)

Your max_length is set to 130, but your input_length is only 28. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=14)


CNN.com will feature iReporter photos in a weekly Travel Snapshots gallery. Visit CNN.com/Travel each week for a new gallery of snapshots from around the world.


### Validación audio

In [17]:
def aparicion_palabras_clave(texto, palabras_clave):
    conteo = {palabra: texto.lower().count(palabra.lower()) for palabra in palabras_clave}
    print("Palabras clave que deben aparecer:", palabras_clave)
    print("Número de palabras clave:", len(palabras_clave))
    print("Conteo de palabras clave que aparecen:", conteo)
    print("Número de palabras clave que aparecen:", sum(1 for count in conteo.values() if count > 0))


##### Audio corto de whatsapp

In [18]:
transcripcion = transcribir_audio("./grabaciones/audiowhatsapp.opus")
print(transcripcion)
resumen = summarize_long_text(transcripcion)
print(resumen)

palabras_clave = ["feria", "nacionales"]

aparicion_palabras_clave(transcripcion, palabras_clave)

Your max_length is set to 130, but your input_length is only 25. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=12)


 Bajaro de los Nacionales volando por arriba de la feria de Sevilla
Bajaro de los Nacionales volando por arriba de la feria de Sevilla. Bajaro   volando  por ariba de the feria.
Palabras clave que deben aparecer: ['feria', 'nacionales']
Número de palabras clave: 2
Conteo de palabras clave que aparecen: {'feria': 1, 'nacionales': 1}
Número de palabras clave que aparecen: 2


#### Letra de canción

In [ ]:
transcripcion = transcribir_audio("./grabaciones/sevillana.mp3")
print(transcripcion)
resumen = summarize_long_text(transcripcion)
print(resumen)

palabras_clave = ["sevilla", "triana", "arte", "niña"]

aparicion_palabras_clave(transcripcion, palabras_clave)

 Cuando paso por el puente triana contigo mi camina  Cuando paso por el puente triana contigo mi camina triana contigo mi camina  Con mirarte solamente mi camina me muero garevia  Porque tienes unos sombianas y cualquier dulceiro triana y cualquier dulceiro  Y una clase de chuva triana te va en un duplero  Si para otro me deja la triana de pena moriría  Me di a nadie, no te quiero y te mere, me rendido el teniría triana triana y un el  Velaita de Santana triana, farones sobre el río  Velaita de Santana triana, farones sobre el río triana  Falo de sobre el río cuantesitos de avellana triana y tuve el plazo río  Un ritano va cantando triana con promos de colores triana con promos de colores  Una pie a fuego una conriana pétiño y alfrajo  Calentito y busco el otro triana que le gra su pestaña  Ay, niña, que mira, mira, me vamos a ver la cumpaña, se te daña triana y un el  Arfarero de triana triana, casi al nos afina  Alfarero de triana triana, casi al nos afina triana, casi al nos afina  

Your max_length is set to 130, but your input_length is only 78. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=39)


Para mirarte solamente mi camina me muero garevia. Porque tienes unos sombianas y cualquier dulceiro triana. Y una clase de chuva triana te va en un duplero. Si para otro me deja la triana de pena moriría. Un ritano va cantando triana con promos de colores triana with promos of colores. Una pie a fuego una conriana pétiño y alfrajo. Calentito y busco el otro triana that le gra su pestaña. Triana is un semplante agonizante triana igual al that se hito y eseño sanizador o triana. Reponda la dosa ella la crea también tiene que ver un cavitar a tambien triana triana y tuve. Ana, la moscilla triana, the moscillas triana. ana, la luna y asasoma. ana. Ana, the bella, the blanca paloma triana triana and tuve.
Palabras clave que deben aparecer: ['sevilla', 'triana', 'giralda', 'arte', 'hija', 'buleria', 'baile']
Número de palabras clave: 7
Conteo de palabras clave que aparecen: {'sevilla': 0, 'triana': 38, 'giralda': 0, 'arte': 1, 'hija': 0, 'buleria': 0, 'baile': 0}
Número de palabras clave qu

#### Audio de video largo

In [20]:
transcripcion = transcribir_audio("./grabaciones/altozano.mp3")
print(transcripcion)
resumen = summarize_long_text(transcripcion)
print(resumen)
palabras_clave = ["canción", "letra", "Bruno", "Mars", "Miley", "Cyrus", "estribillo", "acorde", "melodía", "flowers"]
aparicion_palabras_clave(transcripcion, palabras_clave)

 A ver, Flowers de My Liz Iris.  Hace de la canción más estremeada en su primera semana en la historia de Spotify.  O sea, le he ido muy bien.  Dicho es, quiero analizar Flowers porque me gusta mucho y quiero analizar la canción  en la que se basa, que me gusta que creo que aún más que es Buenaigua's Your Man de  Rommar.  La canción de Flowers está basada en la letra y en la melodía y en los acordes.  Hay mucha gente hablando de que ha cogido cosas de la letra.  La letra es una respuesta a la de Bruno Mars, pero la melodía también es la misma.  A gente que se dedica a analizar música y siendo no tiene nada que ver melodicamente.  Hay artículos de Billboard diciendo no, no coge ningún elemento melodico, no es verdad.  Es una canción respuesta en todas las maneras de responder letra, melodía y acordes.  Se falta la producción, la producción coge la de Will's Urmite, entonces pues ya estamos  a peñón.  Capítulo 1, lo que todo el mundo sea.  La letra de My Liz Iris dice,  «I can buy myself

Your max_length is set to 130, but your input_length is only 88. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=44)


"Flowers de My Liz Iris" es de la canción más estremeada en su primera semana en la historia of Spotify. "Quiero analizar Flowers porque me gusta mucho" My Liz Iris es una canción respuesta en todas las maneras de responder letra, melodía y acordes. A gente que se dedica a analizar música y siendo no tiene nada que ver melodicamente, no es verdad. Miley: 'I don't ever want to have to need someone again where you feel like without them,  you can't be yourself' Esta letra es en respuesta a cómo se sentía Miley en su relación with su ex.  x, está articulándolo como una respuesta,  a una canción de Bruno Mars. Cantadas desde la perspectiva del chico en la relación, diciendo esto. H get yelled. La melodía y los acordes del estribillo de Miley son la estrofe de  the canción de Bruno Mars. Los acordes son los mismos. Si veinte Bruno adornan algunos acordes metiéndole  a una séptima in medio. Bruno hace.  un poco más, porque está todo más dilatado. Bruno es una esencia también hace la menor re

### Validación de video

In [29]:
transcripcion = transcribir_video("./grabaciones/jordiwild.mp4")
print(transcripcion)
resumen = summarize_long_text(transcripcion)
print(resumen)

palabras_clave_audio = ["catalán", "dedoringa", "hablar"]
palabras_clave_video = ["man", "room", "speaking"]

aparicion_palabras_clave(transcripcion, palabras_clave_audio)
aparicion_palabras_clave(transcripcion, palabras_clave_video)


Your max_length is set to 130, but your input_length is only 128. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=64)


ER: 
Describe el video. ASSISTANT: El video mostra a una persona masculina amb una barba llarga i cabellons llargs, vestida amb una camisa sense mànega. La persona es veu sorrient i es mou la mà per la cara, possiblement per a capturar el moment. La càmera es manté en moviment mentre la persona es mou la mà i es fa una expressió de sorpresa o de felicitat. La persona sembla estar
El video mostra a una persona masculina amb una barba llarga i cabellons llargs. La persona es veu sorrient i es mou la mà per la cara, possiblement per a capturar el moment.
Palabras clave que deben aparecer: ['catalán', 'dedoringa', 'hablar']
Número de palabras clave: 3
Conteo de palabras clave que aparecen: {'catalán': 0, 'dedoringa': 0, 'hablar': 0}
Número de palabras clave que aparecen: 0
Palabras clave que deben aparecer: ['man', 'room', 'speaking']
Número de palabras clave: 3
Conteo de palabras clave que aparecen: {'man': 1, 'room': 0, 'speaking': 0}
Número de palabras clave que aparecen: 1


In [31]:
transcripcion = transcribir_video("./grabaciones/caza.mp4")
print(transcripcion)
resumen = summarize_long_text(transcripcion)
print(resumen)
palabras_clave_video = ["hunt", "prey", "lion", "giraffe", "tree"]
aparicion_palabras_clave(transcripcion, palabras_clave_video)

KeyboardInterrupt: 